In [32]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, GlobalAveragePooling2D, concatenate, BatchNormalization, Conv2D, Activation, Add, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef, confusion_matrix
import boto3
from io import BytesIO
import json
import random

In [3]:
# 2. Load Config
# Load configuration from config.json
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

# Set seed
tf.random.set_seed(config["seed"])

# Load configuration values
bucket_name = config["s3_bucket_name"]
train_labels_path = config["train_labels_path"]
test_labels_path = config["test_labels_path"]
train_prefix = config["output_dir_train"]
test_prefix = config["output_dir_test"]
as_gray = config["as_gray"]
img_rows, img_cols = config["img_rows"], config["img_cols"]
channels = config["in_channel"]
num_classes = config["num_classes"]
batch_size = config["batch_size"]
all_epochs = config["all_epochs"]
input_shape = tuple(config["input_shape"])

# Define input shape
input_img = Input(shape=input_shape)

# Initialize S3 client
s3 = boto3.client('s3')

# Helper function to upload numpy arrays to S3
def upload_to_s3(np_array, bucket, key):
    with BytesIO() as buffer:
        np.save(buffer, np_array)
        buffer.seek(0)
        s3.put_object(Bucket=bucket, Key=key, Body=buffer.getvalue())

# Helper function to load numpy arrays from S3
def load_from_s3(bucket, key):
    obj = s3.get_object(Bucket=bucket, Key=key)
    return np.load(BytesIO(obj['Body'].read()))

In [4]:
# Load train labels from S3
obj = s3.get_object(Bucket=bucket_name, Key=train_labels_path)
train_df = pd.read_excel(BytesIO(obj['Body'].read()))

# Split the data into training and validation sets
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=1203)

# Data Loading and Preprocessing with Improved Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [5]:
validation_datagen = ImageDataGenerator(rescale=1.0/255)

def load_images_from_s3(patient_ids, img_rows, img_cols, bucket_name):
    images = []
    for img_path in patient_ids:
        key = img_path.strip()
        print(f"Attempting to load key: {key}")
        try:
            obj = s3.get_object(Bucket=bucket_name, Key=key)
            img = tf.image.decode_png(obj['Body'].read(), channels=3)
            img = tf.image.resize(img, [img_rows, img_cols])
            images.append(img)
        except s3.exceptions.NoSuchKey as e:
            print(f"Error: Key {key} does not exist.")
    if images:
        return tf.stack(images)
    else:
        raise ValueError("No valid images were loaded.")


In [6]:
# Load CC and MLO views for both left and right breasts for training and validation
x_train_CC = load_images_from_s3(train_df.L_CC_file.values.tolist() + train_df.R_CC_file.values.tolist(), img_rows, img_cols, bucket_name)
x_train_MLO = load_images_from_s3(train_df.L_MLO_file.values.tolist() + train_df.R_MLO_file.values.tolist(), img_rows, img_cols, bucket_name)

x_val_CC = load_images_from_s3(val_df.L_CC_file.values.tolist() + val_df.R_CC_file.values.tolist(), img_rows, img_cols, bucket_name)
x_val_MLO = load_images_from_s3(val_df.L_MLO_file.values.tolist() + val_df.R_MLO_file.values.tolist(), img_rows, img_cols, bucket_name)

# Generate y_train and y_val with the correct number of samples
y_train = tf.keras.utils.to_categorical(train_df.target.values.tolist() * 2, num_classes=2)[:len(x_train_CC)]
y_val = tf.keras.utils.to_categorical(val_df.target.values.tolist() * 2, num_classes=2)[:len(x_val_CC)]

# Print shapes of x and y to verify dimensions
print(f"x_train_CC shape: {x_train_CC.shape}")
print(f"x_train_MLO shape: {x_train_MLO.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_val_CC shape: {x_val_CC.shape}")
print(f"x_val_MLO shape: {x_val_MLO.shape}")
print(f"y_val shape: {y_val.shape}")


Attempting to load key: CMMD-D2/train/PNGs/D2-0407_L_CC.png


W0000 00:00:1730347706.228584    4296 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Attempting to load key: CMMD-D2/train/PNGs/D2-0574_L_CC.png
Attempting to load key: CMMD-D2/train/PNGs/D2-0610_L_CC.png
Attempting to load key: CMMD-D2/train/PNGs/D2-0098_L_CC.png
Attempting to load key: CMMD-D2/train/PNGs/D2-0671_L_CC.png
Attempting to load key: CMMD-D2/train/PNGs/D2-0364_L_CC.png
Attempting to load key: CMMD-D2/train/PNGs/D2-0230_L_CC.png
Attempting to load key: CMMD-D2/train/PNGs/D2-0406_L_CC.png
Attempting to load key: CMMD-D2/train/PNGs/D2-0014_L_CC.png
Attempting to load key: CMMD-D2/train/PNGs/D2-0125_L_CC.png
Attempting to load key: CMMD-D2/train/PNGs/D2-0556_L_CC.png
Attempting to load key: CMMD-D2/train/PNGs/D2-0059_L_CC.png
Attempting to load key: CMMD-D2/train/PNGs/D2-0453_L_CC.png
Attempting to load key: CMMD-D2/train/PNGs/D2-0130_L_CC.png
Attempting to load key: CMMD-D2/train/PNGs/D2-0269_L_CC.png
Attempting to load key: CMMD-D2/train/PNGs/D2-0346_L_CC.png
Attempting to load key: CMMD-D2/train/PNGs/D2-0558_L_CC.png
Attempting to load key: CMMD-D2/train/PN

In [15]:
# Channel Spatial Attention
channel_axis = 1 if tf.keras.backend.image_data_format() == "channels_first" else 3

def channel_attention(input_xs, reduction_ratio=0.125):
    channel = int(input_xs.shape[channel_axis])
    maxpool_channel = tf.keras.layers.GlobalMaxPooling2D()(input_xs)
    maxpool_channel = tf.keras.layers.Reshape((1, 1, channel))(maxpool_channel)
    avgpool_channel = tf.keras.layers.GlobalAveragePooling2D()(input_xs)
    avgpool_channel = tf.keras.layers.Reshape((1, 1, channel))(avgpool_channel)
    Dense_One = tf.keras.layers.Dense(units=int(channel * reduction_ratio), activation='relu', kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')
    Dense_Two = tf.keras.layers.Dense(units=int(channel), activation='relu', kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')

    mlp_1_max = Dense_One(maxpool_channel)
    mlp_2_max = Dense_Two(mlp_1_max)
    mlp_1_avg = Dense_One(avgpool_channel)
    mlp_2_avg = Dense_Two(mlp_1_avg)
    channel_attention_feature = tf.keras.layers.Add()([mlp_2_max, mlp_2_avg])
    channel_attention_feature = tf.keras.layers.Activation('sigmoid')(channel_attention_feature)
    return tf.keras.layers.Multiply()([channel_attention_feature, input_xs])

def spatial_attention(channel_refined_feature):
    maxpool_spatial = tf.keras.layers.Lambda(lambda x: tf.reduce_max(x, axis=3, keepdims=True))(channel_refined_feature)
    avgpool_spatial = tf.keras.layers.Lambda(lambda x: tf.reduce_mean(x, axis=3, keepdims=True))(channel_refined_feature)
    max_avg_pool_spatial = tf.keras.layers.Concatenate(axis=3)([maxpool_spatial, avgpool_spatial])
    return tf.keras.layers.Conv2D(filters=1, kernel_size=(3, 3), padding="same", activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(max_avg_pool_spatial)

def CSA(input_xs, reduction_ratio=0.5):
    channel_refined_feature = channel_attention(input_xs, reduction_ratio=reduction_ratio)
    spatial_attention_feature = spatial_attention(channel_refined_feature)
    refined_feature = tf.keras.layers.Multiply()([channel_refined_feature, spatial_attention_feature])
    return tf.keras.layers.Add()([refined_feature, input_xs])


In [16]:
# Improved Model Definition with Two ResNet50 Base Models for CC and MLO Views
# Input tensors for CC and MLO views
I1 = Input(shape=(img_rows, img_cols, 3))
model1 = ResNet50(weights='imagenet', include_top=False, input_tensor=I1, pooling=None)
MLO_model = Model(inputs=I1, outputs=model1.get_layer('conv4_block6_out').output)


I2 = Input(shape=(img_rows, img_cols, 3))
model2 = ResNet50(weights='imagenet', include_top=False, input_tensor=I2, pooling=None)
for layer in model2.layers:
    layer.name = layer.name + '_2'
CC_model = Model(inputs=I2, outputs=model2.get_layer('conv4_block6_out_2').output)


x_mlo = MLO_model.output
x_cc = CC_model.output

# Combine the outputs from MLO and CC views
c1 = concatenate([x_mlo, x_cc], axis=2)


In [26]:
# Define bottleneck block
def bottleneck_Block(input_tensor, nb_filters, strides=(1, 1), with_conv_shortcut=False):
    (nb_filter1, nb_filter2, nb_filter3) = nb_filters

    x = Conv2D(nb_filter1, (1, 1), strides=strides, kernel_initializer='he_normal')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(nb_filter2, (3, 3), padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(nb_filter3, (1, 1), kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)

    if with_conv_shortcut:
        shortcut = Conv2D(nb_filter3, (1, 1), strides=strides, kernel_initializer='he_normal')(input_tensor)
        shortcut = BatchNormalization()(shortcut)
        x = Add()([x, shortcut])
    else:
        x = Add()([x, input_tensor])

    x = Activation('relu')(x)
    return x


In [35]:
Downsampling_model = Model(digit_input, out_d)
digit_input = Input(shape=(16, 16, 1024))

## intra-modality attention
a1=Self_Attention(1024)(c1)

x1 = tf.keras.layers.Lambda(tf.split, arguments={'axis': 2, 'num_or_size_splits': 2})(a1)
x11,x12=x1[0], x1[1]
h1, w1, c1 = x11.shape[1],x11.shape[2],x11.shape[3]
x11=Reshape((h1, w1, c1))(x11)
x12=Reshape((h1, w1, c1))(x12)

x11=Downsampling_model(x11)
x12=Downsampling_model(x12)

## intra-modality attention
a2=Self_Attention(1024)(x_us)

x2 = bottleneck_Block(a2, nb_filters=[512, 512, 2048], strides=(2, 2), with_conv_shortcut=True)
x2 = bottleneck_Block(x2, nb_filters=[512, 512, 2048])
x2 = bottleneck_Block(x2, nb_filters=[512, 512, 2048])
ccc = concatenate([x11, x12, x2], axis=2)

## inter-modality attention
a3=Self_Attention(2048)(ccc)

x6 = tf.keras.layers.Lambda(tf.split, arguments={'axis': 2, 'num_or_size_splits': 3})(a3)
x61, x62, x63=x6[0], x6[1] ,x6[2]
h6, w6, c6 = x61.shape[1],x61.shape[2],x61.shape[3]
x61=Reshape((h6, w6, c6))(x61)
x62=Reshape((h6, w6, c6))(x62)
x63=Reshape((h6, w6, c6))(x63)
c6 = concatenate([x61, x62, x63], axis=3)

## channel_spatial_attention
x3=CSA(c6)

NameError: name 'digit_input' is not defined

In [ ]:
# Define the final model
output = GlobalAveragePooling2D()(c6)
output = Dense(512, activation="relu")(output)
output = Dropout(0.3)(output)
output = Dense(num_classes, activation='softmax', name='output')(output)
model = Model(inputs=[MLO_model.input, CC_model.input], outputs=output)

In [ ]:
# Define the final model
model = Model(inputs=[MLO_model.input, CC_model.input], outputs=output)

# Compile the model with custom metrics
model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Reduced learning rate for more stable training
    loss='categorical_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)